Media outlets are consistently on the lookout for new and upcoming players, regardless of sport. Young talent, valiant comebacks, rise after injury or any combination of the latter generate consistent media coverage, followed by support for that player and love for the sport. For instance, Serena Williams has grown both fame and worldwide support through her distinguished and transformative career as a tennis icon. She has changed the game through her triumphs and unprecedented adversities. Throughout her career, she has won thousands of dollars in prizes and has been breaking tennis records for decades. In our project, we will be analysing statistics from current tennis players, including their prize sums and the number of seasons they have been playing. We will be creating a classification model to predict a player's global ranking based on how many seasons they’ve played and their total prize money. This in turn will help officials classify a player who is new to the league in comparison to the veterans. The dataset that we have obtained was retrieved from “canvas” and consists of several columns, however our focus will be placed upon “Prize Money” and ”Seasons” to predict current rank. 

Firstly, we are reading the necessary packages for our plots and functions for our regression model. These include ggplot2 and tidyverse. 

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

To read our data, we have tidyed most of it outside of Jupyter as this was more efficient. We have titled our raw data "player_data_initial". Below, we had to ensure that the column names were representative of the information, so those were renamed accoridngly. We have subsequently filtered for the players current rank. total seasons, and prize money. 

After the filter, we have placed 75% of our data into training and 25% to testing, with the predictor being "current rank"

In [2]:
player_data_initial <- read_csv("https://raw.githubusercontent.com/pangus3/DSCI-Group-Project/main/data/player_stats.csv")
player_data_initial

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,Age,Country,Plays,Wikipedia,Current Rank,Best Rank,Name,Backhand,Prize Money,⋯,Facebook,Twitter,Nicknames,Grand Slams,Davis Cups,Web Site,Team Cups,Olympics,Weeks at No. 1,Tour Finals
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378,363 (04-11-2019),Oscar Jose Gutierrez,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326,316 (14-10-2019),Jack Draper,Two-handed,59040.00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178,44 (14-01-2013),Lukas Lacko,Two-handed,3261567.00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
497,23 (14-03-1996),Netherlands,Left-handed,Wikipedia,495,342 (05-08-2019),Gijs Brouwer,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
498,24 (17-05-1995),Ukraine,NA,Wikipedia,419,419 (20-01-2020),Vladyslav Orlov,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
499,22 (26-03-1997),Tunisia,Left-handed,Wikipedia,451,408 (24-12-2018),Aziz Dougaz,Two-handed,61984.00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [3]:
colnames(player_data_initial) <- make.names(colnames(player_data_initial), unique=TRUE)
player_data_edited <- player_data_initial |>
mutate(current_rank = as.integer(Current.Rank)) |>
mutate(prize_money = as.integer(Prize.Money)) |>
select(Seasons, current_rank, prize_money) |>
as_tibble()


player_data <- player_data_edited |>
filter(!is.na(Seasons),!is.na(current_rank), !is.na(prize_money) )
player_data

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”
Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


Seasons,current_rank,prize_money
<dbl>,<int>,<int>
14,178,3261567
2,236,374093
11,183,6091971
⋮,⋮,⋮
1,397,40724
10,5,22132368
2,451,61984


In [4]:
player_data
#Splitting the data into the training and testing sets, with 75% of the data going to training, and 25% of it going to testing. 
player_split <- initial_split(player_data, prop = 0.75, strata = current_rank)  
player_train<- training(player_split)   
player_test<- testing(player_split)


Seasons,current_rank,prize_money
<dbl>,<int>,<int>
14,178,3261567
2,236,374093
11,183,6091971
⋮,⋮,⋮
1,397,40724
10,5,22132368
2,451,61984


In [5]:
#Building a model specification (player_spec) to specify the model and training algorithm. 
player_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
      set_engine("kknn") |>
       set_mode("regression") 
player_spec

K-Nearest Neighbor Model Specification (regression)

Main Arguments:
  neighbors = tune()
  weight_func = rectangular

Computational engine: kknn 


In [6]:
#Recipe specifies the predictors/response and preprocesses the data, so it’s scaled. 
player_recipe <- recipe(current_rank ~ Seasons + prize_money, data = player_train) |>
       step_scale(all_predictors()) |>
       step_center(all_predictors())
player_recipe


Recipe

Inputs:

      role #variables
   outcome          1
 predictor          2

Operations:

Scaling for all_predictors()
Centering for all_predictors()

In [7]:
#creating the fold for cross-validation. Cross validation uses a structured splitting so that each observation is used in a validation set only once
player_vfold<-  vfold_cv(player_train, v = 5, strata = current_rank)

#The workflow combines the model and recipe
player_workflow <- workflow() |>
       add_recipe(player_recipe) |>
   add_model(player_spec)


In [8]:
gridvals <- tibble(neighbors= c(1:100))
player_results <- player_workflow |>
       tune_grid(resamples = player_vfold, grid = gridvals) |>
collect_metrics()
player_results

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,rmse,standard,131.1011915,5,5.88810020,Preprocessor1_Model001
1,rsq,standard,0.2742896,5,0.04425432,Preprocessor1_Model001
2,rmse,standard,117.9361475,5,6.30606540,Preprocessor1_Model002
⋮,⋮,⋮,⋮,⋮,⋮,⋮
99,rsq,standard,0.2911660,5,0.07927050,Preprocessor1_Model099
100,rmse,standard,118.8239900,5,5.14002209,Preprocessor1_Model100
100,rsq,standard,0.2900067,5,0.07889446,Preprocessor1_Model100


In [9]:
player_min<- player_results |>
    filter(.metric == "rmse") |>
  arrange(mean) |> 
    slice(1)

player_min

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
11,rmse,standard,104.4621,5,4.190772,Preprocessor1_Model011


In [10]:
k_min <-player_min |>
      pull(neighbors)

player_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) |>
         set_engine("kknn") |>
          set_mode("regression")

player_best_fit <- workflow() |>
         add_recipe(player_recipe) |>
          add_model(player_best_spec) |>
          fit(data = player_train)

player_summary <- player_best_fit |>
         predict(player_test) |>
           bind_cols(player_test) |>
           metrics(truth = current_rank, estimate = .pred)


player_summary

#Explain what this table and values means!!

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,98.1188185
rsq,standard,0.4641113
mae,standard,82.7173913


In [11]:
#Predicting a new players rank given their prize money and seasons of tennis played
#Jack Black is an up and coming tennis player from planet Mars. He has played tennis in the US for the past 3 seasons and has recieved $145,563 dollars in prize money.
#With this information we want to know roughly what Jack Blacks ranking would be so that we can decide if he is someone we want to bet/sponsor.

new_player_stats <- tibble(Seasons = 3, 
                            prize_money=145563)

#This recipe is using all the player data this time to create the model recipe instead of just the trianing.. but it is using the same spec that found the best k!
player_recipe_all_data <- recipe(current_rank ~ Seasons + prize_money, data = player_data) |>
                        step_scale(all_predictors()) |>
                        step_center(all_predictors())
                     
player_fit_all_data <- workflow() |>
          add_recipe(player_recipe_all_data) |>
          add_model(player_best_spec) |>
          fit(data = player_data)

new_player_predicted <- predict(player_fit_all_data, new_player_stats)

new_player_predicted

#He would have a current_rank of 294.6667 I believe!
##EXPLAIN WHAT THIS MEANS FOR US!

.pred
<dbl>
282.1818


In [ ]:
player_plot <- player_data |>
    ggplot(aes(x = current_rank)) +
    geom_histogram(color = "white",
                  fill = "blue",
                  binwidth = 1) +
    facet_grid(prize_money ~ Seasons) +
    labs(x = "Current Rank") +
    ggtitle("Current Rank by Seasons and Prize Money")
           
player_plot

In [ ]:
annotate("path", 
            x=point1[1] + 5*cos(seq(0,2*pi,length.out=100)),
            y=point1[2] + 0.1*sin(seq(0,2*pi,length.out=100))) +
        annotate("text", x = 183, y =  8.5, label = "1")